In [1]:
# Imports

import json
import geojson
import geopandas
import pandas as pd
import pathlib
import csv
import copy
import ast
import os
import time
from collections import OrderedDict

from geojson import Feature, Point, FeatureCollection

MAP_VERSION = 3
STATS_TYPES = ['Count','Percent','nshft', 'pshft','growth', 'Labels']


In [2]:
fips_codes = {
 '01': {'abbr': 'AL', 'full_name': 'alabama'},
 '02': {'abbr': 'AK', 'full_name': 'alaska'},
 '04': {'abbr': 'AZ', 'full_name': 'arizona'},
 '05': {'abbr': 'AR', 'full_name': 'arkansas'},
 '06': {'abbr': 'CA', 'full_name': 'california'},
 '08': {'abbr': 'CO', 'full_name': 'colorado'},
 '09': {'abbr': 'CT', 'full_name': 'connecticut'},
 '10': {'abbr': 'DE', 'full_name': 'delaware'},
 '11': {'abbr': 'DC', 'full_name': 'district_of_columbia'},
 '12': {'abbr': 'FL', 'full_name': 'florida'},
 '13': {'abbr': 'GA', 'full_name': 'georgia'},
 '15': {'abbr': 'HI', 'full_name': 'hawaii'},
 '16': {'abbr': 'ID', 'full_name': 'idaho'},
 '17': {'abbr': 'IL', 'full_name': 'illinois'},
 '18': {'abbr': 'IN', 'full_name': 'indiana'},
 '19': {'abbr': 'IA', 'full_name': 'iowa'},
 '20': {'abbr': 'KS', 'full_name': 'kansas'},
 '21': {'abbr': 'KY', 'full_name': 'kentucky'},
 '22': {'abbr': 'LA', 'full_name': 'louisiana'},
 '23': {'abbr': 'ME', 'full_name': 'maine'},
 '24': {'abbr': 'MD', 'full_name': 'maryland'},
 '25': {'abbr': 'MA', 'full_name': 'massachusetts'},
 '26': {'abbr': 'MI', 'full_name': 'michigan'},
 '27': {'abbr': 'MN', 'full_name': 'minnesota'},
 '28': {'abbr': 'MS', 'full_name': 'mississippi'},
 '29': {'abbr': 'MO', 'full_name': 'missouri'},
 '30': {'abbr': 'MT', 'full_name': 'montana'},
 '31': {'abbr': 'NE', 'full_name': 'nebraska'},
 '32': {'abbr': 'NV', 'full_name': 'nevada'},
 '33': {'abbr': 'NH', 'full_name': 'new_hampshire'},
 '34': {'abbr': 'NJ', 'full_name': 'new_jersey'},
 '35': {'abbr': 'NM', 'full_name': 'new_mexico'},
 '36': {'abbr': 'NY', 'full_name': 'new_york'},
 '37': {'abbr': 'NC', 'full_name': 'north_carolina'},
 '38': {'abbr': 'ND', 'full_name': 'north_dakota'},
 '39': {'abbr': 'OH', 'full_name': 'ohio'},
 '40': {'abbr': 'OK', 'full_name': 'oklahoma'},
 '41': {'abbr': 'OR', 'full_name': 'oregon'},
 '42': {'abbr': 'PA', 'full_name': 'pennsylvania'},
 '44': {'abbr': 'RI', 'full_name': 'rhode_island'},
 '45': {'abbr': 'SC', 'full_name': 'south_carolina'},
 '46': {'abbr': 'SD', 'full_name': 'south_dakota'},
 '47': {'abbr': 'TN', 'full_name': 'tennessee'},
 '48': {'abbr': 'TX', 'full_name': 'texas'},
 '49': {'abbr': 'UT', 'full_name': 'utah'},
 '50': {'abbr': 'VT', 'full_name': 'vermont'},
 '51': {'abbr': 'VA', 'full_name': 'virginia'},
 '53': {'abbr': 'WA', 'full_name': 'washington'},
 '54': {'abbr': 'WV', 'full_name': 'west_virginia'},
 '55': {'abbr': 'WI', 'full_name': 'wisconsin'},
 '56': {'abbr': 'WY', 'full_name': 'wyoming'}}

In [3]:
STATE_FILES = [
    "tl_2010_01_bg10.json",
    "tl_2010_02_bg10.json",
    "tl_2010_04_bg10.json",
    "tl_2010_05_bg10.json",
    "tl_2010_06_bg10.json",
    "tl_2010_08_bg10.json",
    "tl_2010_09_bg10.json",
    "tl_2010_10_bg10.json",
    "tl_2010_11_bg10.json",
    "tl_2010_12_bg10.json",
    "tl_2010_13_bg10.json",
    "tl_2010_15_bg10.json",
    "tl_2010_16_bg10.json",
    "tl_2010_17_bg10.json",
    "tl_2010_18_bg10.json",
    "tl_2010_19_bg10.json",
    "tl_2010_20_bg10.json",
    "tl_2010_21_bg10.json",
    "tl_2010_22_bg10.json",
    "tl_2010_23_bg10.json",
    "tl_2010_24_bg10.json",
    "tl_2010_25_bg10.json",
    "tl_2010_26_bg10.json",
    "tl_2010_27_bg10.json",
    "tl_2010_28_bg10.json",
    "tl_2010_29_bg10.json",
    "tl_2010_30_bg10.json",
    "tl_2010_31_bg10.json",
    "tl_2010_32_bg10.json",
    "tl_2010_33_bg10.json",
    "tl_2010_34_bg10.json",
    "tl_2010_35_bg10.json",
    "tl_2010_36_bg10.json",
    "tl_2010_37_bg10.json",
    "tl_2010_38_bg10.json",
    "tl_2010_39_bg10.json",
    "tl_2010_40_bg10.json",
    "tl_2010_41_bg10.json",
    "tl_2010_42_bg10.json",
    "tl_2010_44_bg10.json",
    "tl_2010_45_bg10.json",
    "tl_2010_46_bg10.json",
    "tl_2010_47_bg10.json",
    "tl_2010_48_bg10.json",
    "tl_2010_49_bg10.json",
    "tl_2010_50_bg10.json",
    "tl_2010_51_bg10.json",
    "tl_2010_53_bg10.json",
    "tl_2010_54_bg10.json",
    "tl_2010_55_bg10.json",
    "tl_2010_56_bg10.json",
]

In [4]:
# Input Files
# 2 => "02"
def formatted_state_fips_code(fips_number):
    return f'{fips_number:02d}'

#  block_group_geojson_filename(state_fips_code=46, year=2010)
# tl_2010_46_bg10.json
def block_group_geojson_filename(state_fips_code=1, census_year=2010):
    year_string = str(census_year)
    fips_code_string = formatted_state_fips_code(state_fips_code)
    return f'tl_{year_string}_{fips_code_string}_bg10.json'

# wa_washington_zip_codes_geo.min.json
# def zip_code_geojson_filename(state_fips_code=1):
#     formatted_fips_code = formatted_state_fips_code(state_fips_code)
#     state_abbr = fips_codes[formatted_fips_code]["abbr"].lower()
#     state_full_name = fips_codes[formatted_fips_code]["full_name"]
#     return f'{state_abbr}_{state_full_name}_zip_codes_geo.min.json'


# def zip_geojson_files_for_market(market_data):
#     fips_codes = json.loads(market_data['StateFips'])
#     files = []
#     for fips_code in fips_codes:
#         files.append(zip_code_geojson_filename(fips_code))
#     return files

def bg_geojson_files_for_market(market_data, census_year):
    fips_codes = json.loads(market_data['StateFips'])
    files = []
    for fips_code in fips_codes:
        files.append(block_group_geojson_filename(state_fips_code=int(fips_code), census_year=census_year))
    return files

def read_geo_gson_file(file_path):
    with open(file_path) as f:
        gj = geojson.load(f)
        features = gj['features']
    print(f'Loaded {len(features)} features from {file_path}')
    return  features

def read_geojson_files(file_paths, base_path = "./"):
    features = []
    for file_path in file_paths:
        features += read_geo_gson_file(os.path.join(base_path, file_path))
    return features
    

# def read_market_zip_code_usage(market_file_prefix, base_path="./source_data" ):
#     data_type = "zip"
#     csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
#     data_rows = []
#     with open(csv_file_path, encoding='utf-8-sig') as csvfile:
#         reader = csv.DictReader(csvfile)
#         for row in reader:
#             data_rows.append(row)
#     zip_codes = []
#     for data_row in data_rows:
#         zip_codes.append(data_row['Zip'].zfill(5))
#     print(f'Loaded {len(zip_codes)} zip_codes from {csv_file_path}')
#     return zip_codes
        
def read_market_block_group_usage(market_file_prefix, base_path="./source_data" ):
    data_type = "bg"
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type=data_type)
    data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    block_groups = []
    for data_row in data_rows:
        block_groups.append(data_row['BG'].zfill(12))
    print(f'Loaded {len(block_groups)} block groups from {csv_file_path}')
    return block_groups
        
# market_file_prefix = 'DallasFortWorth'
# market_prefix= 'dfw'
# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes

def file_type_for_data_type(data_type = "bg"):
    if data_type == "zip": 
        return "Zip"
    if data_type == "bg":
        return "BG"

# DallasFortWorth__Mapping_BG_Data.csv
# ./source_data/{market_file_prefix}__Mapping_BG_Data.csv - the actual zip code data
# 
# DallasFortWorth__Mapping_Zip_Data.csv
# ./source_data/{market_file_prefix}__Mapping_Zip_Data.csv - the actual block group data
# 
def mapping_data_file_path(market_file_prefix, base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_{file_type}_Data.csv'

# DallasFortWorth__Mapping_Settings_Transposed File_BG.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_BG.csv - segment definitions for Block Groups
# DallasFortWorth__Mapping_Settings_Transposed File_Zip.csv
# ./source_data/{market_file_prefix}__Mapping_Settings_Transposed_File_Zip.csv - segment definitions for Zip Codes
def map_settings_data_file_path(market_file_prefix, base_path="./source_data", data_type = "bg"):
    file_type = file_type_for_data_type(data_type = data_type)
    return f'{base_path}/{market_file_prefix}__Mapping_Settings_Transposed File_{file_type}.csv'

def published_map_data_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}/data/{market_prefix}_{data_type}_data_v{MAP_VERSION}.json'

def published_map_settings_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_{data_type}_settings_data_v{MAP_VERSION}.json'

def published_map_labels_path(market_prefix,  data_type = "bg", published_data_base_path="./public"):
    return f'{published_data_base_path}/{market_prefix}_labels_v{MAP_VERSION}.json'

def read_mapping_data(market_file_prefix, base_path='./source_data', data_type='bg'):
    data_rows = []
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path) as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')
    return data_rows

def create_zip_features(features, zip_data_rows):
    new_features_to_keep = []
    for data_row in zip_data_rows:
        zip_code = data_row['Zip'].zfill(5)
        # find the associated feature
        for feature in features:
            if feature['properties']["ZCTA5CE10"] == zip_code:
                new_feature = copy.deepcopy(feature)
                for key in data_row.keys():
                    if key != 'Zip':
                        if data_row[key]:
                            if data_row[key] != 'inf':
                                new_feature['properties'][key] =  ast.literal_eval(data_row[key].replace(',',''))
                new_features_to_keep.append(new_feature)
                break
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_bg_features(features, map_settings_data, bg_data_rows, stats_type):
    new_features_to_keep = []
    for data_row in bg_data_rows:
        block_group = str(data_row['BG']).zfill(12)
        # find the associated feature
        found = False
        if block_group in features:
            found = True
            feature = features[block_group]
            new_feature = copy.deepcopy(feature)
            properties = new_feature['properties']
            properties.pop("ALAND10",0)
            properties.pop("AWATER10",0)
            properties.pop("BLKGRPCE10",0)
            properties.pop("COUNTYFP10",0)
            properties.pop("FUNCSTAT10",0)
            properties.pop("MTFCC10",0)
            properties.pop("NAMELSAD10",0)
            properties.pop("STATEFP10",0)
            properties.pop("TRACTCE10",0)
            new_feature['properties'] = properties
            for key in list(data_row.keys())[0:]:
                if key not in  ['BG','LAT','LONG'] and stats_type in key: #and 'Count' in key:
                    if data_row[key] and data_row[key] != 'inf':
                        translated_key = map_settings_data[key]['mapbox_segment']
                        value = ast.literal_eval(data_row[key].replace(',',''))
                        value = int(value * 1000000) / 1000000
                        new_feature['properties'][translated_key] =  value
            new_features_to_keep.append(new_feature)
    #         if found == False:
    #             print(f'failed: {block_group}')
    print(f'Processed {len(new_features_to_keep)} features to use')
    return new_features_to_keep

def create_geojson(features):
    print(f'Created GeoJson with  {len(features)} features')
    return FeatureCollection(features)


def create_data_json(features):
    data = []
    for feature in features:
        #centroid = {
        #    "GEOID10": feature["properties"]["GEOID10"],
        #    "INTPTLAT10": feature["properties"]["INTPTLAT10"],
        #    "INTPTLON10": feature["properties"]["INTPTLON10"],
        #}
        datum = feature["properties"]
        data.append(datum)
    return data

def write_geojson(geojson_data, mapbox_market_file_prefix, data_type='bg', stats_type='Count',published_data_base_path = "./public"):
    geojson_string = geojson.dumps(geojson_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{mapbox_market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geojson data to  {published_data_base_path}/{mapbox_market_file_prefix}/data/{mapbox_market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json')
    
def write_data_json(json_data, mapbox_market_file_prefix, data_type='bg', stats_type='count',published_data_base_path = "./public"):
    json_string = json.dumps(json_data, sort_keys=True)
    f = open(f'{published_data_base_path}/{mapbox_market_file_prefix}_{stats_type.lower()}_data_v{MAP_VERSION}.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote centroid json data to  {published_data_base_path}/{mapbox_market_file_prefix}/data/{mapbox_market_file_prefix}_{stats_type.lower()}_data_v{MAP_VERSION}.json')


def read_map_settings(market_file_prefix, base_path="./source_data", data_type = "bg"):
    data_rows = []
    csv_file_path = map_settings_data_file_path(market_file_prefix, base_path=base_path, data_type = data_type)
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)
    print(f'Loaded {len(data_rows)} data rows from {csv_file_path}')   
    return data_rows

def write_settings_data(settings_data, mapbox_market_file_prefix, base_path="./public", data_type = "bg"):
    processed_settings_data = OrderedDict()
    segment_number = 0
    for row in settings_data:
        segment_number += 1
        if row["Display_Data_Type"] == "HH_Count":
            row["Display_Data_Type"] = "HH Count"
        mapbox_segment_key = f's{segment_number}'
        row['mapbox_segment'] = mapbox_segment_key
        processed_settings_data[row['UniqueSeriesString']] = row
    json_string = json.dumps(processed_settings_data, indent = 2, sort_keys=False)
    settings_data_file_path = published_map_settings_path(mapbox_market_file_prefix,  data_type = data_type, published_data_base_path=base_path)
    f = open(settings_data_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote settings data rows to {settings_data_file_path}') 
    return processed_settings_data
    

def create_bg_labels(market_file_prefix, mapbox_market_file_prefix, base_path="./source_data" , data_type = "bg" , published_data_base_path="./public"):
    data_rows = []
    #     csv_file_path = './DFW_Mapping_data.csv'
    csv_file_path = mapping_data_file_path(market_file_prefix, base_path=base_path, data_type = "bg")
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data_rows.append(row)

    features = []
    for data_row in data_rows:
        feature =  { 
            "type": "Feature", 
            "properties": {
                "id": data_row['BG']
            },
            "geometry": { 
                "type": "Point", 
                "coordinates": [ float(data_row['LONG']), float(data_row['LAT']) ]
            }
        }

        features.append(feature)
    geo_json = {
        "type": "FeatureCollection",
        "features": features
    }
    json_string = json.dumps(geo_json, indent = 2, sort_keys=True)
    output_file_path = published_map_labels_path(mapbox_market_file_prefix,  data_type , published_data_base_path=published_data_base_path)
    f = open(output_file_path, "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson labels data to  {output_file_path}')

def create_geojson_settings(csv_file_path, published_data_base_path = "./public"):

    geospatial_settings = OrderedDict()
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            row['CenterLat'] = float(row['CenterLat'])
            row['CenterLong'] = float(row['CenterLong'])
            geospatial_settings[row['MarketName']] = row
    geojson_string = json.dumps(geospatial_settings, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/geospatial_settings_v{MAP_VERSION}.json', "w")
    f.write(geojson_string)
    f.close()
    print(f'Wrote geospatials settings data to  {published_data_base_path}/geospatial_settings_v{MAP_VERSION}.json')
    
    
def create_tileset_data_recipe_file(mapbox_market_file_prefix, published_data_base_path, stats_type):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{mapbox_market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{mapbox_market_file_prefix}_{stats_type.lower()}_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
          'tiles': {
            'layer_size': 2500
          }
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{mapbox_market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
    

def create_tileset_labels_recipe_file(mapbox_market_file_prefix, published_data_base_path):
    recipe_file_contents = {
      'version': 1,
      'layers': {
        f'{mapbox_market_file_prefix}_labels_v{MAP_VERSION}': {
          'source': f'mapbox://tileset-source/stobieb/{mapbox_market_file_prefix}_labels_src_v{MAP_VERSION}',
          'minzoom': 0,
          'maxzoom': 13,
            
        }
      }
    }

    recipe_file_path = os.path.join(published_data_base_path,f'{mapbox_market_file_prefix}_labels_recipe_v{MAP_VERSION}.json' )
    with open(recipe_file_path, 'w') as recipe_file:
        recipe_file.write(json.dumps(recipe_file_contents, indent=4))
        
# def mstds_convert_file(market_file_prefix, published_data_base_path, filetype='data'):
#     recipe_file_path = os.path.join(published_data_base_path, market_file_prefix,'labels','mts-recipe.json' )
#     market_file_prefix, published_data_base_path
    

In [5]:
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
def create_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    nationwide_features = {}
    for file in STATE_FILES:
        print(f'processing: {file}')
        geosjon_path = os.path.join(geojson_files_base_path,file)
        features = read_geo_gson_file(geosjon_path)
        for feature in features:
            key = feature['properties']["GEOID10"]
            nationwide_features[key] = feature
    json_string = json.dumps(nationwide_features, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote geojson data to  {geojson_files_base_path}/nationwide_bg.json')

def read_nation_bg_features_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    # Opening JSON file
    f = open(geosjon_path)
    # returns JSON object as 
    # a dictionary
    features = json.load(f)
    # Closing file
    f.close()
    return features

def write_nation_bg_centroids_json(geojson_files_base_path='/Users/alanmccann/Dropbox/bain/map_source_data'):
    geosjon_path = os.path.join(geojson_files_base_path,"nationwide_bg.json")
    features = read_nation_bg_features_json(geojson_files_base_path)
    centroids = []
    for key in features.keys:
        feature = features[key]
        centroid = {
            "GEOID10": key,
            "INTPTLAT10": feature["properties"]["INTPTLAT10"],
            "INTPTLON10": feature["properties"]["INTPTLON10"],
        }
        centroids.append(centroid)
    json_string = json.dumps(centroids, sort_keys=True)
    f = open(f'{geojson_files_base_path}/nationwide_bg_centroids.json', "w")
    f.write(json_string)
    f.close()
    print(f'Wrote bg centroids data to  {geojson_files_base_path}/nationwide_bg_centroids.json')
    
        
        
    

In [6]:
def create_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        # segment = row['Segment']
        year = row['Year']
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(region,year)
        print(urban, suburban, exurban)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        if region not in data_json.keys():
            data_json[region] = {'displayRegion': display_region, 'data': {}}
        if year not in data_json[region]['data'].keys():
            data_json[region]['data'][year] = {}
        if 'Urban' not in data_json[region]['data'][year].keys():
            data_json[region]['data'][year]['Urban'] = {}
            data_json[region]['data'][year]['SubUrban'] = {}
            data_json[region]['data'][year]['ExUrban'] = {}
        data_json[region]['data'][year]['Urban'] = urban
        data_json[region]['data'][year]['SubUrban'] = suburban
        data_json[region]['data'][year]['ExUrban'] = exurban
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_summary_charts_v{MAP_VERSION}.json')



In [7]:
def create_comparison_chart_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    data_json = {}
    for row in chart_data_rows:
        region = row['Region']
        display_region = row['DisplayRegion']
        year = row['Year']
        age_segment=row['Display_Age_Segment']
        income_segment=row['Display_Income_Segment']
        key = f'{region}-{year}-{age_segment}-{income_segment}'
        urban = row['Urban']
        suburban = row['SubUrban']
        exurban = row['ExUrban']
        print(key)
        urban = '%.10f' % float(row['Urban'])
        suburban = '%.10f' % float(row['SubUrban'])
        exurban = '%.10f' % float(row['ExUrban'])
        display_region = row['DisplayRegion']
        isMetro = row['IsMetro']
        data_json[key] = {
            "year": year,
            "ageSegment": age_segment,
            "incomeSegment": income_segment,
            "urban": urban,
            "suburban": suburban,
            "exurban": exurban,
            "isMetro": isMetro,
        }
    chart_data_string = json.dumps(data_json, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json', "w")
    f.write(chart_data_string)
    f.close()
    print(f'Wrote market summary charts data to  {published_data_base_path}/market_comparison_charts_v{MAP_VERSION}.json')
    

In [8]:
def create_data_options_json(published_data_base_path = "./public"):
    csv_file_path = f'{source_data_base_path}/Chart_Data_File.csv'
    chart_data_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            chart_data_rows.append(row)
    year_options_set = set()
    age_segment_options_set = set()
    income_segment_options_set = set()
    region_options_list = []
    for row in chart_data_rows:
        year_options_set.add(row["Year"])
        age_segment_options_set.add(row["Display_Age_Segment"])
        income_segment_options_set.add(row["Display_Income_Segment"])
        region = {
            "region": row["Region"],
            "displayRegion": row["DisplayRegion"],
            "isMetro": row["IsMetro"],
        }
        region_options_list.append(region)
    year_options_list = list(year_options_set)
    year_options_list.sort()
    age_segment_options_list = list(age_segment_options_set)
    age_segment_options_list.sort()
    income_segment_options_list = list(income_segment_options_set)
    income_segment_options_list.sort()
    region_options = pd.DataFrame(region_options_list).drop_duplicates().to_dict('r')
    dataOptions = {
        "yearOptions": year_options_list,
        'ageSegmentOptions': age_segment_options_list,
        'incomeSegmentOptions': income_segment_options_list,
        "regionOptions": region_options
    }
    data_options_string = json.dumps(dataOptions, indent=4, sort_keys=True)
    f = open(f'{published_data_base_path}/data_options_v{MAP_VERSION}.json', "w")
    f.write(data_options_string)
    f.close()
    print(f'Wrote data options data to  {published_data_base_path}/data_options_v{MAP_VERSION}.json')


In [1]:
def process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year):
    nationwide_bg_features = read_nation_bg_features_json(geojson_files_base_path)
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows:
        #         if market_data['MarketName'] in ['HiltonHeadIslandSC','HomosassaSpringsFL', 'DistrictOfColumbia','CharlottesvilleVA','ColoradoSpringsCO','DallasFortWorthTX','EastStroudsburgPA']:
        #             #if True:
        #             #print(market_data['MarketName'])
        if market_data['MarketName'] in ['HiltonHeadIslandSC','HomosassaSpringsFL', 'DistrictOfColumbia','CharlottesvilleVA','ColoradoSpringsCO','DallasFortWorthTX','EastStroudsburgPA']:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MarketName'].lower()
            mapbox_market_file_prefix =  market_data['MapboxMarketName'].lower()
            market_prefix = market_file_prefix

            for stats_type in STATS_TYPES:
                create_tileset_data_recipe_file(mapbox_market_file_prefix, published_data_base_path,stats_type)
                create_tileset_labels_recipe_file(mapbox_market_file_prefix, published_data_base_path)
            # Block Group Processing
            data_type = 'bg'
            map_settings_data = read_map_settings(market_file_prefix, base_path=source_data_base_path, data_type = "bg")
            map_settings = write_settings_data(map_settings_data, mapbox_market_file_prefix, base_path=published_data_base_path, data_type = "bg")
            block_group_rows = read_mapping_data(market_file_prefix, source_data_base_path, data_type)
            for stats_type in STATS_TYPES:
                features_to_use = create_bg_features(nationwide_bg_features, map_settings, block_group_rows, stats_type)
                new_geojson = create_geojson(features_to_use)
                write_geojson(new_geojson, mapbox_market_file_prefix, data_type, stats_type,published_data_base_path=published_data_base_path)
                if stats_type.lower() != 'labels':
                    new_data_json = create_data_json(features_to_use)
                    write_data_json(new_data_json, mapbox_market_file_prefix.lower(),data_type, stats_type.lower(),published_data_base_path)
            create_bg_labels(market_file_prefix, mapbox_market_file_prefix, base_path=source_data_base_path , data_type = "bg" , published_data_base_path=published_data_base_path)
            market_count += 1
    # Finished states - create overall settings file    
    #create_geojson_settings(csv_file_path, published_data_base_path = "./public")
    print(f'*** COMPLETED {market_count} markets')
    print(completed_markets)
                          
        
# source_data_base_path = "./source_data"
# source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/v5_9th_run Brian Stobie/"
# published_data_base_path = "./public"
# geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# census_year = 2010

In [2]:
# Set up input and output locations
# Geojson file path
geojson_files_base_path = '/Users/alanmccann/Dropbox/bain/map_source_data'
# Source csv data path
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/2022_3rd_run Brian Stobie"
# Output data path - all of the files in this folder should be uploaded to the cms GeoSpatialTool/v1 folder
published_data_base_path = "./public"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
# Census year for geojson
census_year = 2010


# Process the data
process_main_geo_data(source_data_base_path, geojson_files_base_path, published_data_base_path, census_year)
# create_chart_json(published_data_base_path)
# create_comparison_chart_json(published_data_base_path)
# create_data_options_json(published_data_base_path)

NameError: name 'read_nation_bg_features_json' is not defined

In [10]:
def upload_to_mapbox(source_data_base_path,published_data_base_path):
    csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
    market_rows = []
    with open(csv_file_path, encoding='utf-8-sig') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            market_rows.append(row)
    market_count = 0
    completed_markets = []
    for market_data in market_rows: #[0:10]:
        # if market_data['MarketName'] in ['Texas','DallasFortWorth']:
        if True:
            print("")
            print(f'Market Data: {market_count + 1}')
            market_file_prefix = market_data['MapboxMarketName'].lower()
            os.system(f'cd {published_data_base_path}; tilesets upload-source stobieb {market_file_prefix}_bg_data-src_v{MAP_VERSION} {market_file_prefix}_bg_data_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION} --recipe {market_file_prefix}-bg-recipe_v{MAP_VERSION}.json --name "{market_file_prefix}_bg_data"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_data_v{MAP_VERSION}')
            os.system(f'cd {published_data_base_path};  tilesets upload-source stobieb {market_file_prefix}_bg_labels-src {market_file_prefix}_bg_labels_v{MAP_VERSION}.json')
            os.system(f'cd {published_data_base_path};  tilesets create stobieb.{market_file_prefix}_bg_labels --recipe {market_file_prefix}-labels-recipe.json --name "{market_file_prefix}_bg_labels_v{MAP_VERSION}"')
            os.system(f'cd {published_data_base_path};  tilesets publish stobieb.{market_file_prefix}_bg_labels')

In [11]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}

import subprocess
# tilesets job stobieb.dallasfortworth_bg_data cl9kgdofo000209l52zyy3mdi
process = subprocess.Popen(['tilesets', 'job', 'stobieb.dallasfortworth_bg_data', 'cl9kgdofo000209l52zyy3mdi'],
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)
stdout, stderr = process.communicate()
output = stdout.decode('utf-8').split('\n')
error = stderr.decode('utf-8')
result = json.loads(output[0])



In [12]:
import os
import subprocess
import json
MAPBOX_ACCESS_TOKEN='sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'
my_env = {**os.environ, 'MAPBOX_ACCESS_TOKEN':'sk.eyJ1Ijoic3RvYmllYiIsImEiOiJjbDR2dDlibHkwODhjM2lub3EwOXJld2dzIn0.Ur59yGDIvj60ELA7QOCnqQ'}
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
import subprocess

cwd = os.getcwd()
os.chdir(published_data_base_path)

market_file_prefix = 'austin'
stats_type = 'Count'

def upload_source(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_{stats_type.lower()}_src_v{MAP_VERSION}',
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}.json',
        '--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)

def upload_label_source(market_file_prefix, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
    upload_source_command = [
        'tilesets',
        'upload-source',
        'stobieb',
        f'{market_file_prefix}_labels_src_v{MAP_VERSION}',
        f'{market_file_prefix}_labels_v{MAP_VERSION}.json',
        #'--replace'
    ]
    print(' '.join(upload_source_command))
    process = subprocess.Popen(upload_source_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output)
    
# upload_source(market_file_prefix, stats_type, published_data_base_path)

In [13]:
def update_recipe(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    update_recipe_command = [
        'tilesets',
        'update-recipe',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        
    ]

    print(' '.join(update_recipe_command))
    process = subprocess.Popen(update_recipe_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    
def upload_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        '--recipe', 
        f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

def upload_label_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    create_tileset_command = [
        'tilesets',
        'create',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        '--recipe', f'{market_file_prefix}_{stats_type.lower()}_recipe_v{MAP_VERSION}.json',
        '--name',
        f'{market_file_prefix}_{stats_type.lower()}'
        
    ]

    print(' '.join(create_tileset_command))
    process = subprocess.Popen(create_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)

In [14]:
import time
def publish_tileset(market_file_prefix, stats_type, working_directory):
    cwd = os.getcwd()
    os.chdir(working_directory)
                # print(f'tilesets create stobieb.{market_file_prefix}_bg_{stats_type.lower()}_data --recipe {market_file_prefix}_bg_{stats_type.lower()}_recipe.json --name "{market_file_prefix}_bg_{stats_type.lower()}_data"')
    publish_tileset_command = [
        'tilesets',
        'publish',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}'
    ]

    print(' '.join(publish_tileset_command))
    print(datetime.datetime.now())
    process = subprocess.Popen(publish_tileset_command,
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE,
                         env=my_env)

    stdout, stderr = process.communicate()
    output = stdout.decode('utf-8').split('\n')
    error = stderr.decode('utf-8')
    print(output, error)
    result = json.loads(output[0])
    print(result)
    get_job_status_command = [
        'tilesets',
        'job',
        f'stobieb.{market_file_prefix}_{stats_type.lower()}_v{MAP_VERSION}',
        result['jobId']
    ]
    state = 'processing'
    while state == 'processing':
        process = subprocess.Popen(get_job_status_command,
                     stdout=subprocess.PIPE, 
                     stderr=subprocess.PIPE,
                     env=my_env)

        stdout, stderr = process.communicate()
        output = stdout.decode('utf-8').split('\n')
        try:
            result = json.loads(output[0])
        except:
            result = {}
        if 'stage' in result.keys():
            state = result['stage']
        if state != 'success':
            print(f'{state}: sleeping')
            time.sleep(15)
    print('finished')
    print(datetime.datetime.now())


In [18]:
import csv
import datetime
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/2022_3rd_run Brian Stobie"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)
market_count = 0
completed_markets = []
STATS_TYPES = ['growth'] #['count','percent','nshft','labels','pshft']
# STATS_TYPES = ['nshft']
for market_data in market_rows[215:]: #[339:]:
    #market_data['MarketName'] in ['HiltonHeadIslandSC','HomosassaSpringsFL', 'DistrictOfColumbia','CharlottesvilleVA','ColoradoSpringsCO','DallasFortWorthTX','EastStroudsburgPA']:
    #market_file_prefix = market_data['MapboxMarketName'].lower()
    #if True: #market_data['MarketName'] in ['Texas','DallasFortWorthTX']:
    if True: #market_data['MarketName'] in ['HiltonHeadIslandSC', 'HomosassaSpringsFL', 'DistrictOfColumbia','CharlottesvilleVA','ColoradoSpringsCO','DallasFortWorthTX','EastStroudsburgPA']:
        market_file_prefix = market_data['MapboxMarketName'].lower()
        print('creating tilesets for:', market_file_prefix)
        print(datetime.datetime.now())
        for stats_type in STATS_TYPES:
            upload_source(market_file_prefix, stats_type, published_data_base_path)
            upload_tileset(market_file_prefix, stats_type, published_data_base_path)
            update_recipe(market_file_prefix, stats_type, published_data_base_path)
            publish_tileset(market_file_prefix, stats_type, published_data_base_path)
        
        

creating tilesets for: madisonwi
2023-05-01 19:40:06.618647
tilesets upload-source stobieb madisonwi_growth_src_v3 madisonwi_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/madisonwi_growth_src_v3", "files": 1, "source_size": 3050098, "file_size": 3050098}', '']
tilesets create stobieb.madisonwi_growth_v3 --recipe madisonwi_growth_recipe_v3.json --name madisonwi_growth_v3
['{"message": "stobieb.madisonwi_growth_v3 already exists"}', ''] 
tilesets update-recipe stobieb.madisonwi_growth_v3 madisonwi_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.madisonwi_growth_v3
2023-05-01 19:40:14.076617
['{"message": "Processing stobieb.madisonwi_growth_v3", "jobId": "clh5hgv23000408ju1vg96un8"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.madisonwi_growth_v3 or run tilesets job stobieb.madisonwi_growth_v3 clh5hgv23000408ju1vg96un8 to view the status of your tileset.

{'message': 'Processing stobieb.madiso

[''] Traceback (most recent call last):
  File "/Users/alanmccann/opt/anaconda3/bin/tilesets", line 8, in <module>
    sys.exit(cli())
  File "/Users/alanmccann/opt/anaconda3/lib/python3.9/site-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/Users/alanmccann/opt/anaconda3/lib/python3.9/site-packages/click/core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "/Users/alanmccann/opt/anaconda3/lib/python3.9/site-packages/click/core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/Users/alanmccann/opt/anaconda3/lib/python3.9/site-packages/click/core.py", line 1395, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/Users/alanmccann/opt/anaconda3/lib/python3.9/site-packages/click/core.py", line 754, in invoke
    return __callback(*args, **kwargs)
  File "/Users/alanmccann/opt/anaconda3/lib/python3.9/site-packages/mapbox_tilesets/scripts/cli.py", line 496, in update_recipe

finished
2023-05-01 19:59:34.895311
creating tilesets for: midlandtx
2023-05-01 19:59:34.895856
tilesets upload-source stobieb midlandtx_growth_src_v3 midlandtx_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/midlandtx_growth_src_v3", "files": 1, "source_size": 448854, "file_size": 448854}', '']
tilesets create stobieb.midlandtx_growth_v3 --recipe midlandtx_growth_recipe_v3.json --name midlandtx_growth_v3
['{"message": "Successfully created empty tileset stobieb.midlandtx_growth_v3. Publish your tileset to begin processing your data into vector tiles."}', ''] 
tilesets update-recipe stobieb.midlandtx_growth_v3 midlandtx_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.midlandtx_growth_v3
2023-05-01 19:59:41.407210
['{"message": "Processing stobieb.midlandtx_growth_v3", "jobId": "clh5i5vg2001108l37ce91gfw"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.midlandtx_growth_v3 or run tilesets job stob

['{"message": "Successfully created empty tileset stobieb.monroela_growth_v3. Publish your tileset to begin processing your data into vector tiles."}', ''] 
tilesets update-recipe stobieb.monroela_growth_v3 monroela_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.monroela_growth_v3
2023-05-01 20:09:34.226064
['{"message": "Processing stobieb.monroela_growth_v3", "jobId": "clh5iiky0004x08mi8ja4fhbm"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.monroela_growth_v3 or run tilesets job stobieb.monroela_growth_v3 clh5iiky0004x08mi8ja4fhbm to view the status of your tileset.

{'message': 'Processing stobieb.monroela_growth_v3', 'jobId': 'clh5iiky0004x08mi8ja4fhbm'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-05-01 20:10:59.922769
creating tilesets for: monroemi
2023-05-01 20:10:59.922922
tilesets upload-source stobieb monroemi_growth_src_v3 monroemi_growth_v3.jso

['{"message": "Processing stobieb.munciein_growth_v3", "jobId": "clh5iv95q003908l392cf1k75"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.munciein_growth_v3 or run tilesets job stobieb.munciein_growth_v3 clh5iv95q003908l392cf1k75 to view the status of your tileset.

{'message': 'Processing stobieb.munciein_growth_v3', 'jobId': 'clh5iv95q003908l392cf1k75'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-05-01 20:21:23.734106
creating tilesets for: muskegonmi
2023-05-01 20:21:23.734206
tilesets upload-source stobieb muskegonmi_growth_src_v3 muskegonmi_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/muskegonmi_growth_src_v3", "files": 1, "source_size": 390014, "file_size": 390014}', '']
tilesets create stobieb.muskegonmi_growth_v3 --recipe muskegonmi_growth_recipe_v3.json --name muskegonmi_growth_v3
['{"me

...

[''] Updated recipe.

tilesets publish stobieb.michigan_growth_v3
2023-05-02 01:00:13.678163
['{"message": "Processing stobieb.michigan_growth_v3", "jobId": "clh5swdqr000508l3cz3md6lf"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.michigan_growth_v3 or run tilesets job stobieb.michigan_growth_v3 clh5swdqr000508l3cz3md6lf to view the status of your tileset.

{'message': 'Processing stobieb.michigan_growth_v3', 'jobId': 'clh5swdqr000508l3cz3md6lf'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-05-02 01:01:58.835134
creating tilesets for: minnesota
2023-05-02 01:01:58.835294
tilesets upload-source stobieb minnesota_growth_src_v3 minnesota_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/minnesota_growth_src_v3", "files": 1, "source_size": 32816429, "file_size": 32816429}', '']
tilesets create stobieb.minnesota_growth_v3 --rec

processing: sleeping
finished
2023-05-02 01:12:24.440838
creating tilesets for: newhampshire
2023-05-02 01:12:24.441420
tilesets upload-source stobieb newhampshire_growth_src_v3 newhampshire_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/newhampshire_growth_src_v3", "files": 1, "source_size": 5909066, "file_size": 5909066}', '']
tilesets create stobieb.newhampshire_growth_v3 --recipe newhampshire_growth_recipe_v3.json --name newhampshire_growth_v3
['{"message": "Successfully created empty tileset stobieb.newhampshire_growth_v3. Publish your tileset to begin processing your data into vector tiles."}', ''] 
tilesets update-recipe stobieb.newhampshire_growth_v3 newhampshire_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.newhampshire_growth_v3
2023-05-02 01:12:35.625931
['{"message": "Processing stobieb.newhampshire_growth_v3", "jobId": "clh5tcaig000408mb2wci8b8a"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/ti

[''] Updated recipe.

tilesets publish stobieb.ohio_growth_v3
2023-05-02 01:53:27.943648
['{"message": "Processing stobieb.ohio_growth_v3", "jobId": "clh5ustw0000p08mmdjq9dglb"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.ohio_growth_v3 or run tilesets job stobieb.ohio_growth_v3 clh5ustw0000p08mmdjq9dglb to view the status of your tileset.

{'message': 'Processing stobieb.ohio_growth_v3', 'jobId': 'clh5ustw0000p08mmdjq9dglb'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-05-02 01:55:09.375575
creating tilesets for: oklahoma
2023-05-02 01:55:09.375707
tilesets upload-source stobieb oklahoma_growth_src_v3 oklahoma_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/oklahoma_growth_src_v3", "files": 1, "source_size": 27600646, "file_size": 27600646}', '']
tilesets create stobieb.oklahoma_growth_v3 --recipe oklahoma_growth_recip

processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-05-02 02:04:45.593174
creating tilesets for: tennessee
2023-05-02 02:04:45.593308
tilesets upload-source stobieb tennessee_growth_src_v3 tennessee_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/tennessee_growth_src_v3", "files": 1, "source_size": 57418553, "file_size": 57418553}', '']
tilesets create stobieb.tennessee_growth_v3 --recipe tennessee_growth_recipe_v3.json --name tennessee_growth_v3
['{"message": "Successfully created empty tileset stobieb.tennessee_growth_v3. Publish your tileset to begin processing your data into vector tiles."}', ''] 
tilesets update-recipe stobieb.tennessee_growth_v3 tennessee_growth_recipe_v3.json
[''] Updated recipe.

tilesets publish stobieb.tennessee_growth_v3
2023-05-02 02:05:10.876659
['{"message": "Processing stobieb.tennessee_growth_v3", "jobId": "clh5v7wn8000n08mjezuw9a7v"}', ''] 
✔ Tilese

[''] Updated recipe.

tilesets publish stobieb.westvirginia_growth_v3
2023-05-02 02:15:08.454260
['{"message": "Processing stobieb.westvirginia_growth_v3", "jobId": "clh5vkpn1000808jq4bjt5cwb"}', ''] 
✔ Tileset job received. Visit https://studio.mapbox.com/tilesets/stobieb.westvirginia_growth_v3 or run tilesets job stobieb.westvirginia_growth_v3 clh5vkpn1000808jq4bjt5cwb to view the status of your tileset.

{'message': 'Processing stobieb.westvirginia_growth_v3', 'jobId': 'clh5vkpn1000808jq4bjt5cwb'}
processing: sleeping
processing: sleeping
processing: sleeping
processing: sleeping
finished
2023-05-02 02:16:19.498736
creating tilesets for: wisconsin
2023-05-02 02:16:19.498873
tilesets upload-source stobieb wisconsin_growth_src_v3 wisconsin_growth_v3.json --replace
['upload progress', '{"id": "mapbox://tileset-source/stobieb/wisconsin_growth_src_v3", "files": 1, "source_size": 34921054, "file_size": 34921054}', '']
tilesets create stobieb.wisconsin_growth_v3 --recipe wisconsin_growth_r

In [2]:
import csv
source_data_base_path = "/Users/alanmccann/Dropbox/bain/bain-uploads/2022_3rd_run Brian Stobie"
published_data_base_path = '/Users/alanmccann/Dropbox/bain/mtsds-data'
csv_file_path = f'{source_data_base_path}/Market_Mapping_Settings.csv'
market_rows = []
with open(csv_file_path, encoding='utf-8-sig') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        market_rows.append(row)

In [5]:
market_rows[9]

{'MarketName': 'AmarilloTX',
 'Market_Display_Name': 'Amarillo, TX',
 'StateFips': '',
 'CenterLat': '35.207215999999995',
 'CenterLong': '-101.83098199999999',
 'GeoTypeGroup': '1 - Designated Metro Areas',
 'DefaultZoomLevel': '8',
 'InterMarketMetro': '0'}